In [1]:
data = read.csv("/home/ubuntu/covid19-severity-prediction/county_csv/0_Autauga_36047", head = TRUE)

In [2]:
data

X,Days,S,I,D
<int>,<int>,<dbl>,<int>,<int>
0,0,2582830,0,0
1,1,2582830,0,0
2,2,2582830,0,0
3,3,2582830,0,0
4,4,2582830,0,0
5,5,2582830,0,0
6,6,2582830,0,0
7,7,2582830,0,0
8,8,2582830,0,0


In [3]:
library(MultiBD)

In [4]:
loglik_sir <- function(param, data) {
    alpha <- exp(param[1]) # Rates must be non-negative 
    beta <- exp(param[2])

    # Set-up SIR model
    drates1 <- function(a, b) { 0 }
    brates2 <- function(a, b) { 0 }
    drates2 <- function(a, b) { alpha * b     }
    trans12 <- function(a, b) { beta  * a * b }

    sum(sapply(1:(nrow(data) - 1), # Sum across all time steps k 
        function(k) {
            log(
                dbd_prob( # Compute the transition probability matrix
                    t = data$time[k + 1] - data$time[k], # Time increment
                    a0 = data$S[k], b0 = data$I[k], # From: S(t_k), I(t_k) 
                    drates1, brates2, drates2, trans12,
                    a = data$S[k + 1], B = data$S[k] + data$I[k] - data$S[k + 1], 
                    computeMode = 4, nblocks = 80 # Compute using 4 threads
                )[1, data$I[k + 1] + 1] # To: S(t_(k+1)), I(t_(k+1)) 
            )
        }))
}

In [5]:
logprior <- function(param) {
    log_alpha <- param[1]
    log_beta <- param[2]
    
    dnorm(log_alpha, mean = 0, sd = 100, log = TRUE) + dnorm(log_beta, mean = 0, sd = 100, log = TRUE)
}

In [9]:
# source("http://bioconductor.org/biocLite.R")
biocLite("graph")
biocLite("Rgraphviz")
# install.packages("MCMCpack", repos = 'http://cran.us.r-project.org') 
library(MCMCpack)

ERROR: Error in biocLite("graph"): could not find function "biocLite"


In [8]:
alpha0 <- 3.39
beta0  <- 0.0212

In [13]:
post_sample <- MCMCmetrop1R(fun = function(param) { loglik_sir(param, data) + logprior(param)},
                        theta.init = log(c(alpha0, beta0)),
                           mcmc = 1000, burnin = 200)

ERROR: Error in MCMCmetrop1R(fun = function(param) {: could not find function "MCMCmetrop1R"


In [10]:
alpha <- exp(post_sample[,1])
beta <- exp(post_sample[,2])

ERROR: Error in eval(expr, envir, enclos): object 'post_sample' not found


In [1]:
1+1

[1] 2